# Capstone Project

In [71]:
#Import libraries
import pandas as pd
import numpy as np
from urllib.request import urlopen
# Import BeautifulSoup from bs4
from bs4 import BeautifulSoup

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [72]:
#Store the url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)

In [73]:
# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, "lxml")
type(soup)

bs4.BeautifulSoup

In [74]:
#Get the text
text = soup.get_text()
#print(text)

From these object extract the table data

In [75]:
# Print the first 10 rows 
rows = soup.find_all('tr')
print(rows[:10])

[<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>, <tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>, <tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>, <tr>
<td>M6A</td>
<td><a href="/wiki/North_York" ti

In [76]:
#Create a loop that iterate through table rows and print out the cells of the rows
for row in rows:
    row_td = row.find_all('td')
print(row_td)
type(row_td)

[<td align="center" style="border: 1px solid #FF0000; background-color: #FFE0E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_A" title="List of postal codes of Canada: A">A</a>
</td>, <td align="center" style="border: 1px solid #FF4000; background-color: #FFE8E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_B" title="List of postal codes of Canada: B">B</a>
</td>, <td align="center" style="border: 1px solid #FF8000; background-color: #FFF0E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_C" title="List of postal codes of Canada: C">C</a>
</td>, <td align="center" style="border: 1px solid #FFC000; background-color: #FFF8E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_E" title="List of postal codes of Canada: E">E</a>
</td>, <td align="center" style="border: 1px solid #FFFF00; background-color: #FFFFE0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_code

bs4.element.ResultSet

In [77]:
#Use beautiful soup to to remove the html tags
str_cells = str(row_td)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()
print(cleantext)

[A
, B
, C
, E
, G
, H
, J
, K
, L
, M
, N
, P
, R
, S
, T
, V
, X
, Y
]


In [78]:
import re

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)

[A
, B
, C
, E
, G
, H
, J
, K
, L
, M
, N
, P
, R
, S
, T
, V
, X
, Y
]


str

In [79]:
df = pd.DataFrame(data = list_rows)
df.head(10)

,0
0,[]
1,"[M1A, Not assigned, Not assigned\n]"
2,"[M2A, Not assigned, Not assigned\n]"
3,"[M3A, North York, Parkwoods\n]"
4,"[M4A, North York, Victoria Village\n]"
5,"[M5A, Downtown Toronto, Harbourfront\n]"
6,"[M5A, Downtown Toronto, Regent Park\n]"
7,"[M6A, North York, Lawrence Heights\n]"
8,"[M6A, North York, Lawrence Manor\n]"
9,"[M7A, Queen's Park, Not assigned\n]"


In [80]:
df1 = df[0].str.split(',', expand=True)
df1.head(10)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,[],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,[M1A,Not assigned,Not assigned\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,[M2A,Not assigned,Not assigned\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,[M3A,North York,Parkwoods\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,[M4A,North York,Victoria Village\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,[M5A,Downtown Toronto,Harbourfront\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,[M5A,Downtown Toronto,Regent Park\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,[M6A,North York,Lawrence Heights\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,[M6A,North York,Lawrence Manor\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,[M7A,Queen's Park,Not assigned\n],None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [86]:
df1 = df1.ix[1:, :2]

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [87]:
df1[0] = df1[0].str.strip('[')
df1.head(10)

,0,1,2
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


In [88]:
df1[2] = df1[2].str.strip('\n]')
df1.head(10)

,0,1,2
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


In [89]:
df1.columns = ["Postcode", "Borough", "Neighborhood"]


In [90]:
df1.head()

,Postcode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [91]:
df=df1[df1['Borough']!='Not assigned']

In [92]:
df1.loc[df1['Neighborhood']=='Not assigned', ['Neighborhood']]=df1['Borough']

In [93]:
geodata = pd.read_csv("https://cocl.us/Geospatial_data")
geodata.rename(columns = {'Postal Code' : 'Postcode'}, inplace = True)
geodata.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [94]:
#Merge dataframes
df= pd.merge(df1,geodata,
                 on='Postcode', 
                 how='left')
df = df.dropna()

In [98]:
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
6,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [99]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 212 neighborhoods.
